# Attention Ensemble - 앙상블에 사용할 개별 모델
---

### **Import Libraries**

In [124]:
import tensorflow as tf
from keras.models import Model
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
from sklearn.model_selection import train_test_split 

import unicodedata
import re
import numpy as np
import os
import io
import time
import random
import openpyxl

### **데이터 로드**

In [125]:
path_to_file_esb = '/Users/ahjeong_park/Study/Attention-Ensemble-Translation/spa-eng/spa_for_esb.txt'

### **데이터 랜덤 셔플**


*   [영어, 스페인어] 쌍 shuffle file 새로 저장
*   번역 테스트 시 주석 처리



In [126]:
# lines = io.open(path_to_file, encoding='UTF-8').read().strip().split('\n')
# random.shuffle(lines)
# f = open(path_to_file_esb, 'w')
# for i in lines:
#     data = i + '\n'
#     f.write(data)
# f.close()

### **데이터(문장) 전처리**

In [127]:
# 유니코드 파일을 아스키 코드 파일로 변환합니다.
def unicode_to_ascii(s):
  return ''.join(c for c in unicodedata.normalize('NFD', s)
      if unicodedata.category(c) != 'Mn')


def preprocess_sentence(w):
  w = unicode_to_ascii(w.lower().strip())

  # 단어와 단어 뒤에 오는 구두점(.)사이에 공백을 생성합니다.
  # 예시: "he is a boy." => "he is a boy ."
  # 참고:- https://stackoverflow.com/questions/3645931/python-padding-punctuation-with-white-spaces-keeping-punctuation
  w = re.sub(r"([?.!,¿])", r" \1 ", w)
  w = re.sub(r'[" "]+', " ", w)

  # (a-z, A-Z, ".", "?", "!", ",")을 제외한 모든 것을 공백으로 대체합니다.
  w = re.sub(r"[^a-zA-Z?.!,¿]+", " ", w)

  w = w.strip()

  # 모델이 예측을 시작하거나 중단할 때를 알게 하기 위해서
  # 문장에 start와 end 토큰을 추가합니다.
  w = '<start> ' + w + ' <end>'
  return w

### **Dataset 생성**
1. 문장에 있는 억양을 제거합니다.
2. 불필요한 문자를 제거하여 문장을 정리합니다.
3. 다음과 같은 형식으로 문장의 쌍을 반환합니다: [영어, 스페인어]

In [128]:
def create_dataset(path, num_examples):
  lines = io.open(path, encoding='UTF-8').read().strip().split('\n')

  word_pairs = [[preprocess_sentence(w) for w in l.split('\t')]  for l in lines[:num_examples]]

  return zip(*word_pairs)

### **Language 가 들어오면 공백 단위로 토큰화**
- fit_on_texts(): 문자 데이터를 입력받아서 리스트의 형태로 변환
- texts_to_sequences: 텍스트 안의 단어들을 숫자 시퀀스로 출력
- pad_sequcences(tensor, padding='post') : 서로 다른 개수의 단어로 이루어진 문장을 같은 길이로 만들어주기 위해 패딩을 사용
  - padding = 'post' : [[ 0  0  0  5  3  2  4], [ 0  0  0  5  3  2  7],...,]
  - padding = 'pre' : 뒤 부터 패딩이 채워짐
  - 가장 긴 sequence 의 길이 만큼
  

In [129]:
def tokenize(lang):
  lang_tokenizer = tf.keras.preprocessing.text.Tokenizer(
      filters='')
  lang_tokenizer.fit_on_texts(lang)

  tensor = lang_tokenizer.texts_to_sequences(lang)

  tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor,
                                                         padding='post')

  return tensor, lang_tokenizer

### **전처리된 타겟 문장과 입력 문장 쌍을 생성**
- input_tensor : input 문장의 패딩 처리된 숫자 시퀀스
- inp_lang_tokenizer : input 문장을 공백 단위로 토큰화, 문자 -> 리스트 변환
- target_tensor, targ_lang_tokenizer : 위와 비슷


In [130]:
def load_dataset(path, num_examples=None):
  
  targ_lang, inp_lang = create_dataset(path, num_examples)

  input_tensor, inp_lang_tokenizer = tokenize(inp_lang)
  target_tensor, targ_lang_tokenizer = tokenize(targ_lang)

  return input_tensor, target_tensor, inp_lang_tokenizer, targ_lang_tokenizer

### **언어 데이터셋 크기 제한**
- 언어 데이터셋을 아래의 크기로 제한하여 훈련과 검증을 수행
- inp_lang, targ_lang : 인풋,타겟 문장의 문자 -> 리스트 변환 결과
- max_length_targ, max_length_inp : 인풋, 타겟 문장의 '패딩된' 숫자 시퀀스 길이 -> 타겟 텐서와 입력 텐서의 최대 길이

In [131]:
# num_examples = 30000
num_examples = 60000
input_tensor, target_tensor, inp_lang, targ_lang = load_dataset(path_to_file_esb, num_examples)
# input_tensor2, target_tensor2, inp_lang2, targ_lang2 = load_dataset(path_to_file, 1, num_examples)
# input_tensor3, target_tensor3, inp_lang3, targ_lang3 = load_dataset(path_to_file, 2, num_examples)


max_length_targ, max_length_inp = target_tensor.shape[1], input_tensor.shape[1]
# max_length_targ2, max_length_inp2 = target_tensor2.shape[1], input_tensor2.shape[1]
# max_length_targ3, max_length_inp3 = target_tensor3.shape[1], input_tensor3.shape[1]

### **데이터셋 (테스트 & 검증) 분리**

In [132]:
# 훈련 집합과 검증 집합을 80대 20으로 분리합니다.
input_tensor_train, input_tensor_val, target_tensor_train, target_tensor_val = train_test_split(input_tensor, target_tensor, test_size=0.2)

# 훈련 집합과 검증 집합의 데이터 크기를 출력합니다.
print(len(input_tensor_train), len(target_tensor_train), len(input_tensor_val), len(target_tensor_val))

48000 48000 12000 12000


### 인덱스 -> 해당 word 로

```
Input Language; index to word mapping
1 ----> <start>
93 ----> tomas
27 ----> le
1063 ----> escribio
7 ----> a
120 ----> maria
3 ----> .
2 ----> <end>
```


```
Target Language; index to word mapping
1 ----> <start>
8 ----> tom
695 ----> wrote
6 ----> to
31 ----> mary
3 ----> .
2 ----> <end>
```



In [133]:
def convert(lang, tensor):
  for t in tensor:
    if t!=0:
      print ("%d ----> %s" % (t, lang.index_word[t]))

### **Buffer, Batch, epoch, embedding dimension, units 설정**
- Tokenizer 의 word_index 속성 : 속성은 단어와 숫자의 키-값 쌍을 포함하는 딕셔너리를 반환
- 따라서 vocab_inp_size, vocab_inp_size : 인풋, 타겟의 단어-숫자 딕셔너리 최대 길이 + 1 (?)
- dataset.batch(BATCH_SIZE, drop_remainder = True) : 배치사이즈 만큼 분할 후 남은 데이터를 drop 할 것인지 여부
- shuffle : 데이터셋 적절히 섞어준다.

In [134]:
BUFFER_SIZE = len(input_tensor_train)
BATCH_SIZE = 64
steps_per_epoch = len(input_tensor_train)//BATCH_SIZE
embedding_dim = 256
units = 1024
vocab_inp_size = len(inp_lang.word_index)+1
vocab_tar_size = len(targ_lang.word_index)+1

# 훈련 집합에서만 shuffle, batch
dataset = tf.data.Dataset.from_tensor_slices((input_tensor_train, target_tensor_train)).shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE, drop_remainder=True)

In [135]:
example_input_batch, example_target_batch = next(iter(dataset))
example_input_batch.shape, example_target_batch.shape

(TensorShape([64, 53]), TensorShape([64, 51]))

### **Encoder**


1.   초기화 : vocab_size(단어의 크기), embedding_dim(임베딩 차원 수), enc_units(인코더의 히든 사이즈), batch_sz(배치 사이즈)
  - embedding_dim : 단어 -> 임베딩 벡터로 하기 위한 차원 수
2.  call : gru 에 들어가 output, state 출력
3.  initialize_hidden_state : 맨 처음 gru에 들어가기 위한 더미 입력 값




In [136]:
class Encoder(tf.keras.Model):
  def __init__(self, vocab_size, embedding_dim, enc_units, batch_sz):
    super(Encoder, self).__init__()
    self.batch_sz = batch_sz
    self.enc_units = enc_units
    self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
    self.gru = tf.keras.layers.GRU(self.enc_units,
                                   return_sequences=True,
                                   return_state=True,
                                   recurrent_initializer='glorot_uniform')

  def call(self, x, hidden):
    x = self.embedding(x)
    output, state = self.gru(x, initial_state = hidden)
    return output, state

  def initialize_hidden_state(self):
    return tf.zeros((self.batch_sz, self.enc_units))

### **Encoder 객체 생성**

In [137]:
# encoder = Encoder(vocab_inp_size, embedding_dim, units, BATCH_SIZE)

### **Attention**


In [138]:
class BahdanauAttention(tf.keras.layers.Layer):
  def __init__(self, units):
    super(BahdanauAttention, self).__init__()
    self.W1 = tf.keras.layers.Dense(units)
    self.W2 = tf.keras.layers.Dense(units)
    self.V = tf.keras.layers.Dense(1)

  def call(self, query, values):
    # 쿼리 은닉 상태(query hidden state)는 (batch_size, hidden size)쌍으로 이루어져 있습니다.
    # query_with_time_axis은 (batch_size, 1, hidden size)쌍으로 이루어져 있습니다.
    # values는 (batch_size, max_len, hidden size)쌍으로 이루어져 있습니다.
    # 스코어(score)계산을 위해 덧셈을 수행하고자 시간 축을 확장하여 아래의 과정을 수행합니다.
    query_with_time_axis = tf.expand_dims(query, 1)

    # score는 (batch_size, max_length, 1)쌍으로 이루어져 있습니다.
    # score를 self.V에 적용하기 때문에 마지막 축에 1을 얻습니다.
    # self.V에 적용하기 전에 텐서는 (batch_size, max_length, units)쌍으로 이루어져 있습니다.
    score = self.V(tf.nn.tanh(
        self.W1(query_with_time_axis) + self.W2(values)))

    # attention_weights는 (batch_size, max_length, 1)쌍으로 이루어져 있습니다. 
    attention_weights = tf.nn.softmax(score, axis=1)

    # 덧셈이후 컨텍스트 벡터(context_vector)는 (batch_size, hidden_size)쌍으로 이루어져 있습니다.
    context_vector = attention_weights * values
    context_vector = tf.reduce_sum(context_vector, axis=1)

    return context_vector, attention_weights

### **Decoder**


1.   초기화 : vocab_size(단어의 크기), embedding_dim(임베딩 차원 수), enc_units(인코더의 히든 사이즈), batch_sz(배치 사이즈)
2.   encoder 와의 차이점 : 마지막 fully_connected_layer(tf.keras.layers.Dense) 추가



In [139]:
class Decoder(tf.keras.Model):
  def __init__(self, vocab_size, embedding_dim, dec_units, batch_sz):
    super(Decoder, self).__init__()
    self.batch_sz = batch_sz
    self.dec_units = dec_units
    self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
    self.gru = tf.keras.layers.GRU(self.dec_units,
                                   return_sequences=True,
                                   return_state=True,
                                   recurrent_initializer='glorot_uniform')
    self.fc = tf.keras.layers.Dense(vocab_size)

    # 어텐션을 사용합니다.
    self.attention = BahdanauAttention(self.dec_units)

  def call(self, x, hidden, enc_output):
    # enc_output는 (batch_size, max_length, hidden_size)쌍으로 이루어져 있습니다.
    context_vector, attention_weights = self.attention(hidden, enc_output)

    # 임베딩층을 통과한 후 x는 (batch_size, 1, embedding_dim)쌍으로 이루어져 있습니다.
    x = self.embedding(x)

    # 컨텍스트 벡터와 임베딩 결과를 결합한 이후 x의 형태는 (batch_size, 1, embedding_dim + hidden_size)쌍으로 이루어져 있습니다.
    x = tf.concat([tf.expand_dims(context_vector, 1), x], axis=-1)

    # 위에서 결합된 벡터를 GRU에 전달합니다.
    output, state = self.gru(x)

    # output은 (batch_size * 1, hidden_size)쌍으로 이루어져 있습니다.
    output = tf.reshape(output, (-1, output.shape[2]))

    # output은 (batch_size, vocab)쌍으로 이루어져 있습니다.
    x = self.fc(output)

    # return x, state, attention_weights
    return x, state

### **Decoder 객체 생성**

In [140]:
# decoder = Decoder(vocab_tar_size, embedding_dim, units, BATCH_SIZE)

### **NMT Model 생성**

In [141]:
class NMT_Model():
  def __init__(self):
    super(NMT_Model, self).__init__()
    self.encoder = Encoder(vocab_inp_size, embedding_dim, units, BATCH_SIZE)
    self.decoder = Decoder(vocab_tar_size, embedding_dim, units, BATCH_SIZE)

### **Ensemble Model 생성**

In [142]:
models = []
# num_models = 5
num_models = 6
for m in range(num_models):
  m = NMT_Model()
  models.append(m)

### **Loss Function & Optimizer**

In [143]:
optimizer = tf.keras.optimizers.Adam()
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True, reduction='none')

def loss_function(real, pred):
  mask = tf.math.logical_not(tf.math.equal(real, 0))
  loss_ = loss_object(real, pred)

  mask = tf.cast(mask, dtype=loss_.dtype)
  loss_ *= mask

  return tf.reduce_mean(loss_)

### **Chekcpoint**
- 여기서 학습한 매개변수를 저장, optimizer/encoder/decoder

In [144]:
checkpoint_dir = '/Users/ahjeong_park/Study/Attention-Ensemble-Translation/training_checkpoints_esb'
checkpoint_dir_test = '/Users/ahjeong_park/Study/Attention-Ensemble-Translation/training_checkpoints_esb2'
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt")
checkpoints = []

for m in range(num_models):
  checkpoint = tf.train.Checkpoint(optimizer=optimizer,
                                 encoder=models[m].encoder,
                                 decoder=models[m].decoder)
  checkpoints.append(checkpoint)


In [145]:
print(checkpoints)

[<tensorflow.python.training.tracking.util.Checkpoint object at 0x7ffa4309b630>, <tensorflow.python.training.tracking.util.Checkpoint object at 0x7ff903f081d0>, <tensorflow.python.training.tracking.util.Checkpoint object at 0x7ff903f087b8>, <tensorflow.python.training.tracking.util.Checkpoint object at 0x7ff903f08400>, <tensorflow.python.training.tracking.util.Checkpoint object at 0x7ff903f084a8>, <tensorflow.python.training.tracking.util.Checkpoint object at 0x7ff903f08080>]


### **Train_step**

In [146]:
# @tf.function
def train_step(model, inp, targ, enc_hidden):
  loss = 0

  with tf.GradientTape() as tape:
    enc_output, enc_hidden = model.encoder(inp, enc_hidden)

    dec_hidden = enc_hidden

    dec_input = tf.expand_dims([targ_lang.word_index['<start>']] * BATCH_SIZE, 1)

    # 교사 강요(teacher forcing) - 다음 입력으로 타겟을 피딩(feeding)합니다.
    for t in range(1, targ.shape[1]):
      # enc_output를 디코더에 전달합니다.
      predictions, dec_hidden = model.decoder(dec_input, dec_hidden, enc_output)
      # print('predictions', predictions.shape)

      loss += loss_function(targ[:, t], predictions)

      # 교사 강요(teacher forcing)를 사용합니다. -> 훈련에서는 실제 값을 이용
      dec_input = tf.expand_dims(targ[:, t], 1)

  batch_loss = (loss / int(targ.shape[1]))
  variables = model.encoder.trainable_variables + model.decoder.trainable_variables
  gradients = tape.gradient(loss, variables)
  optimizer.apply_gradients(zip(gradients, variables))
  return batch_loss

### **학습**

In [147]:
# EPOCHS = 10
# model_loss = {0 : 0, 1 : 0, 2 : 0}

# for epoch in range(EPOCHS):
#   start = time.time()

#   total_loss_0 = 0
#   total_loss_1 = 0
#   total_loss_2 = 0
#   enc_hidden_0 = models[0].encoder.initialize_hidden_state()
#   enc_hidden_1 = models[1].encoder.initialize_hidden_state()
#   enc_hidden_2 = models[2].encoder.initialize_hidden_state()

#   # m1_train_step = train_step()
#   # m2_train_step = train_step()
#   for (batch, (inp, targ)) in enumerate(dataset.take(steps_per_epoch)):

#       batch_loss_0 = train_step(models[0], inp, targ, enc_hidden_0)
#       batch_loss_1 = train_step(models[1], inp, targ, enc_hidden_1)
#       batch_loss_2 = train_step(models[0], inp, targ, enc_hidden_2)

#       total_loss_0 += batch_loss_0
#       total_loss_1 += batch_loss_1
#       total_loss_2 += batch_loss_2

#       if batch % 100 == 0:
#         print('Model {} Epoch {} Batch {} Loss {:.4f}'.format(models[0], epoch + 1,
#                                                     batch,
#                                                     batch_loss_0.numpy()))
#         print('Model {} Epoch {} Batch {} Loss {:.4f}'.format(models[1], epoch + 1,
#                                                     batch,
#                                                     batch_loss_1.numpy()))
#         print('Model {} Epoch {} Batch {} Loss {:.4f}'.format(models[2], epoch + 1,
#                                                     batch,
#                                                     batch_loss_2.numpy()))
#   # 각 모델의 최종 loss 를 저장
#   model_loss[0] = total_loss_0
#   model_loss[1] = total_loss_1
#   model_loss[2] = total_loss_2

#   # 에포크가 2번 실행될때마다 모델 저장 (모델 별 체크포인트)
#   if (epoch + 1) % 2 == 0:
#     for idx, checkpoint in enumerate(checkpoints):
#       checkpoint.save(file_prefix=checkpoint_prefix+'-{}'.format(idx))

#   print('Model 1 : Epoch {} Loss {:.4f}'.format(epoch + 1,
#                                       model_loss[0] / steps_per_epoch))
#   print('Model 2 : Epoch {} Loss {:.4f}'.format(epoch + 1,
#                                       model_loss[1] / steps_per_epoch))
#   print('Model 3 : Epoch {} Loss {:.4f}'.format(epoch + 1,
#                                       model_loss[2] / steps_per_epoch))
#   print('Time taken for 1 epoch {} sec\n'.format(time.time() - start))

### **문장 번역(스페인 -> 영어)** 

*   tf.keras.preprocessing.sequence.pad_sequences([inputs], maxlen='', padding='post') : 일정한 길이(maxlen)로 맞춰준다. (패딩은 뒤에서)
*   

  ```
  inp_lang.word_index :  {'<start>': 1, '<end>': 2, '.': 3, 'tom': 4, '?': 5...}
  ```

* tf.expand_dims: 차원을 늘려준다.




In [148]:
def evaluate(sentence):


  sentence = preprocess_sentence(sentence)

  # 문장, input 딕셔너리 출력 
  print ('sentence:', sentence)

  no_word = 'no word'
  inputs = []
  inputs2 = []
  inputs3 = []
  inputs4 = []
  inputs5 = [] 
  inputs6 = [] 
    
  for i in sentence.split(' '):
    if i in inp_lang.word_index:
        inputs.append(inp_lang.word_index[i])
        inputs2.append(inp_lang.word_index[i])
        inputs3.append(inp_lang.word_index[i])
        inputs4.append(inp_lang.word_index[i])
        inputs5.append(inp_lang.word_index[i])
        inputs6.append(inp_lang.word_index[i])
    else:
        print('no words!')
        return no_word, no_word, no_word, no_word, no_word, no_word, sentence
    

  inputs = tf.keras.preprocessing.sequence.pad_sequences([inputs],
                                                         maxlen=max_length_inp,
                                                         padding='post')
  inputs2 = tf.keras.preprocessing.sequence.pad_sequences([inputs2],
                                                         maxlen=max_length_inp,
                                                         padding='post')
  inputs3 = tf.keras.preprocessing.sequence.pad_sequences([inputs3],
                                                         maxlen=max_length_inp,
                                                         padding='post')
  inputs4 = tf.keras.preprocessing.sequence.pad_sequences([inputs4],
                                                         maxlen=max_length_inp,
                                                         padding='post')
  inputs5 = tf.keras.preprocessing.sequence.pad_sequences([inputs5],
                                                         maxlen=max_length_inp,
                                                         padding='post')
  inputs6 = tf.keras.preprocessing.sequence.pad_sequences([inputs6],
                                                         maxlen=max_length_inp,
                                                         padding='post')
    
  inputs = tf.convert_to_tensor(inputs)
  inputs2 = tf.convert_to_tensor(inputs2)
  inputs3 = tf.convert_to_tensor(inputs3)
  inputs4 = tf.convert_to_tensor(inputs4)
  inputs5 = tf.convert_to_tensor(inputs5)
  inputs6 = tf.convert_to_tensor(inputs6)


  result1 = ''
  result2 = ''
  result3 = ''
  result4 = ''
  result5 = ''
  result6 = ''
  voting_result = ''

  hidden = [tf.zeros((1, units))]
  hidden2 = [tf.zeros((1, units))]
  hidden3 = [tf.zeros((1, units))]
  hidden4 = [tf.zeros((1, units))]
  hidden5 = [tf.zeros((1, units))]
  hidden6 = [tf.zeros((1, units))]

  # Encoder 의 hidden 을 Decoder 의 hidden 으로 받는다.
  enc_out, enc_hidden = models[0].encoder(inputs, hidden)

  enc_out2, enc_hidden2 = models[1].encoder(inputs2, hidden2)

  enc_out3, enc_hidden3 = models[2].encoder(inputs3, hidden3)
    
  enc_out4, enc_hidden4 = models[3].encoder(inputs4, hidden4)

  enc_out5, enc_hidden5 = models[4].encoder(inputs5, hidden5)
    
  enc_out6, enc_hidden6 = models[5].encoder(inputs6, hidden6)



  dec_hidden = enc_hidden
  dec_hidden2 = enc_hidden2
  dec_hidden3 = enc_hidden3
  dec_hidden4 = enc_hidden4
  dec_hidden5 = enc_hidden5
  dec_hidden6 = enc_hidden6


  # Decoder 의 시작인 '<start>' 
  dec_input = tf.expand_dims([targ_lang.word_index['<start>']], 0)
  dec_input2 = tf.expand_dims([targ_lang.word_index['<start>']], 0)
  dec_input3 = tf.expand_dims([targ_lang.word_index['<start>']], 0)
  dec_input4 = tf.expand_dims([targ_lang.word_index['<start>']], 0)
  dec_input5 = tf.expand_dims([targ_lang.word_index['<start>']], 0)
  dec_input6 = tf.expand_dims([targ_lang.word_index['<start>']], 0)

  # Target 의 최대 길이 만큼 출력
  for t in range(max_length_inp):
    predictions, dec_hidden = models[0].decoder(dec_input,
                                                         dec_hidden,
                                                         enc_out)
    predictions2, dec_hidden2 = models[1].decoder(dec_input2,
                                                         dec_hidden2,
                                                         enc_out2)
    predictions3, dec_hidden3 = models[2].decoder(dec_input3,
                                                         dec_hidden3,
                                                         enc_out3)
    predictions4, dec_hidden4 = models[3].decoder(dec_input4,
                                                         dec_hidden4,
                                                         enc_out4)
    predictions5, dec_hidden5 = models[4].decoder(dec_input5,
                                                         dec_hidden5,
                                                         enc_out5)
    predictions6, dec_hidden6 = models[5].decoder(dec_input6,
                                                         dec_hidden6,
                                                         enc_out6)
    
    predicted_id = tf.argmax(predictions[0]).numpy() 
    predicted_id2 = tf.argmax(predictions2[0]).numpy() 
    predicted_id3 = tf.argmax(predictions3[0]).numpy() 
    predicted_id4 = tf.argmax(predictions4[0]).numpy() 
    predicted_id5 = tf.argmax(predictions5[0]).numpy()
    predicted_id6 = tf.argmax(predictions6[0]).numpy()

    result1 += targ_lang.index_word[predicted_id] + ' '
    result2 += targ_lang.index_word[predicted_id2] + ' '
    result3 += targ_lang.index_word[predicted_id3] + ' '
    result4 += targ_lang.index_word[predicted_id4] + ' '
    result5 += targ_lang.index_word[predicted_id5] + ' '
    result6 += targ_lang.index_word[predicted_id6] + ' '
    
    if (targ_lang.index_word[predicted_id6] == '<end>') or (targ_lang.index_word[predicted_id5] == '<end>') or (targ_lang.index_word[predicted_id4] == '<end>') or (targ_lang.index_word[predicted_id3] == '<end>') or (targ_lang.index_word[predicted_id2] == '<end>') or (targ_lang.index_word[predicted_id] == '<end>'):
      return result1, result2, result3, result4, result5, result6, sentence


    # 예측된 ID를 모델에 다시 피드합니다. (voting_id)
    dec_input = tf.expand_dims([predicted_id], 0)
    dec_input2 = tf.expand_dims([predicted_id2], 0)
    dec_input3 = tf.expand_dims([predicted_id3], 0)
    dec_input4 = tf.expand_dims([predicted_id4], 0)
    dec_input5 = tf.expand_dims([predicted_id5], 0)
    dec_input6 = tf.expand_dims([predicted_id6], 0)

  # return result, sentence, attention_plot
  return result1, result2, result3, result4, result5, result6, sentence

In [149]:
def translate(sentence):
    result1, result2, result3, result4, result5, result6, sentence = evaluate(sentence)
#     result_list = [('Model 1', result1),
#                    ('Model 2', result2),
#                    ('Model 3', result3),
#                    ('Model 4', result4),
#                    ('Model 5', result5),
#                    ('Model 6', result6)]
#     print('Model 1: ', result1)
#     print('Model 2: ', result2)
#     print('Model 3: ', result3)
#     print('Model 4: ', result4)
#     print('Model 5: ', result5)
#     print('Model 6: ', result6)
    
    return result1, result2, result3, result4, result5, result6


### **Checkpoint 복원**

In [150]:
ckp_dir_m1 = '/Users/ahjeong_park/Study/Survival-Ensemble/Checkpoint/training_checkpoints'
ckp_dir_m2 = '/Users/ahjeong_park/Study/Survival-Ensemble/Checkpoint/training_checkpoints2'
ckp_dir_m3 = '/Users/ahjeong_park/Study/Survival-Ensemble/Checkpoint/training_checkpoints3'
ckp_dir_m4 = '/Users/ahjeong_park/Study/Survival-Ensemble/Checkpoint/training_checkpoints4'
ckp_dir_m5 = '/Users/ahjeong_park/Study/Survival-Ensemble/Checkpoint/training_checkpoints5' 
ckp_dir_m6 = '/Users/ahjeong_park/Study/Survival-Ensemble/Checkpoint/training_checkpoints6'
ckp_dir_m7 = '/Users/ahjeong_park/Study/Survival-Ensemble/Checkpoint/training_checkpoints7'
ckp_dir_m8 = '/Users/ahjeong_park/Study/Survival-Ensemble/Checkpoint/training_checkpoints8'
ckp_dir_m9 = '/Users/ahjeong_park/Study/Survival-Ensemble/Checkpoint/training_checkpoints9'
ckp_dir_m10 = '/Users/ahjeong_park/Study/Survival-Ensemble/Checkpoint/training_checkpoints10'
ckp_dir_m11 = '/Users/ahjeong_park/Study/Survival-Ensemble/Checkpoint/training_checkpoints11'
ckp_dir_m12 = '/Users/ahjeong_park/Study/Survival-Ensemble/Checkpoint/training_checkpoints12'
ckp_dir_m13 = '/Users/ahjeong_park/Study/Survival-Ensemble/Checkpoint/training_checkpoints13'
ckp_dir_m14 = '/Users/ahjeong_park/Study/Survival-Ensemble/Checkpoint/training_checkpoints14'
ckp_dir_m15 = '/Users/ahjeong_park/Study/Survival-Ensemble/Checkpoint/training_checkpoints15' 

In [151]:
# checkpoint_dir내에 있는 최근 체크포인트(checkpoint)를 복원
checkpoints[0].restore(tf.train.latest_checkpoint(ckp_dir_m1))
checkpoints[1].restore(tf.train.latest_checkpoint(ckp_dir_m2))
checkpoints[2].restore(tf.train.latest_checkpoint(ckp_dir_m3))
checkpoints[3].restore(tf.train.latest_checkpoint(ckp_dir_m4))
checkpoints[4].restore(tf.train.latest_checkpoint(ckp_dir_m5))
checkpoints[5].restore(tf.train.latest_checkpoint(ckp_dir_m6))
checkpoints[6].restore(tf.train.latest_checkpoint(ckp_dir_m7))
checkpoints[7].restore(tf.train.latest_checkpoint(ckp_dir_m8))
checkpoints[8].restore(tf.train.latest_checkpoint(ckp_dir_m9))
checkpoints[9].restore(tf.train.latest_checkpoint(ckp_dir_m10))
checkpoints[10].restore(tf.train.latest_checkpoint(ckp_dir_m11))
checkpoints[11].restore(tf.train.latest_checkpoint(ckp_dir_m12))
checkpoints[12].restore(tf.train.latest_checkpoint(ckp_dir_m13))
checkpoints[13].restore(tf.train.latest_checkpoint(ckp_dir_m14))
checkpoints[14].restore(tf.train.latest_checkpoint(ckp_dir_m15))

### 이 코드로 했을 때 학습 바로 돌렸을 때와 같은 결과가 나왔음.

### **번역 시작**

In [152]:
# test dataset 파일 불러오기
testdata = '/Users/ahjeong_park/Study/Survival-Ensemble/Dataset/test_data.txt'
start_row = 2

# 엑셀 파일 불러오기
wb = openpyxl.load_workbook('Survival_Translate.xlsx')

# 엑셀 파일의 시트 활성화
sheet1 = wb['model_1']
sheet2 = wb['model_2']
sheet3 = wb['model_3']
sheet4 = wb['model_4']
sheet5 = wb['model_5']
sheet6 = wb['model_6']
    
f = open(testdata, 'r')
lines = f.readlines()

for sentence in lines:
    result1, result2, result3, result4, result5, result6 = translate(sentence)
    sheet1.cell(row = start_row, column = 4).value = result1
    sheet2.cell(row = start_row, column = 4).value = result2
    sheet3.cell(row = start_row, column = 4).value = result3
    sheet4.cell(row = start_row, column = 5).value = result4
    sheet5.cell(row = start_row, column = 6).value = result5
    sheet6.cell(row = start_row, column = 7).value = result6
    start_row += 1
    
f.close()
wb.save('Survival_Translate.xlsx')

sentence: <start> te dijeron lo que paso , ¿ no ? <end>
sentence: <start> ellos no estaban alla . <end>
sentence: <start> no me gusta ninguno de los chicos . <end>
sentence: <start> jamas trabaje con el . <end>
sentence: <start> mi padre cultiva arroz . <end>
sentence: <start> dime que hiciste en shounan . <end>
sentence: <start> ¿ cuantas palabras deberias escribir ? <end>
sentence: <start> ¿ como te introdujiste en mi casa ? <end>
sentence: <start> mi madre hornea pan todas las mananas . <end>
sentence: <start> yo he disfrutado el leer esta novela . <end>
sentence: <start> el estaba parado en la esquina . <end>
sentence: <start> el testigo no parecia estar nervioso cuando testifico en el juicio . <end>
sentence: <start> ¿ donde esta el zoologico ? <end>
sentence: <start> el nombre de mi hijo es tom . <end>
sentence: <start> tenemos cosas mas grandes de que preocuparnos . <end>
sentence: <start> juntemonos pasado manana . <end>
sentence: <start> ¿ que estas escribiendo ? <end>
sentenc

sentence: <start> estoy cansado de todo este fastidio . <end>
sentence: <start> tus manos estan frias . <end>
sentence: <start> prosigamos . <end>
sentence: <start> no fueron capaces de descubrir ningun secreto . <end>
sentence: <start> tomas es muy manoso . <end>
no words!
sentence: <start> no es de buena educacion hablar con la boca llena . <end>
sentence: <start> yo me siento adormitado . <end>
no words!
sentence: <start> creo que mirar television es una perdida de tiempo . <end>
sentence: <start> la reina elizabeth i fallecio en . <end>
no words!
sentence: <start> el ha escrito muchas historias . <end>
sentence: <start> de todos los libros publicados ultimamente , solo algunos merecen ser leidos . <end>
no words!
sentence: <start> necesito regresar . <end>
sentence: <start> probate este pullover . <end>
no words!
sentence: <start> tom actua como si ni conociera a maria . <end>
sentence: <start> es mi ultima oferta . <end>
sentence: <start> no me voy a casar nunca . <end>
sentence: 

sentence: <start> las polillas son atraidas por la luz . <end>
no words!
sentence: <start> deberiamos dejar descansar a los perros dormidos . <end>
sentence: <start> no se si queremos hacerlo . <end>
sentence: <start> creo que fue un malentendido . <end>
sentence: <start> es tu deber terminar el trabajo . <end>
sentence: <start> ¿ me podrian dar un trozo de tarta de queso ? <end>
sentence: <start> dile a tom que no estare alli . <end>
sentence: <start> no hables con la boca llena . <end>
sentence: <start> me costo conseguir una entrada para el concierto . <end>
sentence: <start> nos vemos una vez por mes . <end>
sentence: <start> es uno de los libros mas conocidos de la literatura brasilera . <end>
no words!
sentence: <start> tom y mary han estado divorciados por mas de tres anos . ¿ en serio ? no lo sabia . <end>
sentence: <start> tenemos que encontrar una nueva ninera . <end>
sentence: <start> tenemos que prevenir a tom . <end>
sentence: <start> se que todos ustedes son unos cobardes

sentence: <start> quiero cantar la cancion . <end>
sentence: <start> tom trato de matarse . <end>
no words!
sentence: <start> tom parece muy incomodo . <end>
sentence: <start> mi celular esta sonando . <end>
sentence: <start> esta melodia les resultara familiar a muchos japoneses . <end>
sentence: <start> ella va a una escuela para sordos . <end>
sentence: <start> el sabe como hablarles a los ninos . <end>
no words!
sentence: <start> tom sabe unos cuantos trucos de magia . <end>
sentence: <start> pidele ayuda a tu padre . <end>
sentence: <start> le ignore . <end>
sentence: <start> creo que es una buena idea . <end>
sentence: <start> quiero divertirme . <end>
sentence: <start> si yo tuviera alas , volaria hasta ti . <end>
no words!
sentence: <start> nos conocimos en la facultad . <end>
sentence: <start> no te molestes en contestar esta carta . <end>
sentence: <start> manana va a haber una carrera de tres millas . <end>
sentence: <start> tengo que cerrar las ventanas . <end>
sentence: <s

sentence: <start> tom conserva un par de zapatos extra en el maletero de su auto . <end>
sentence: <start> tom estaba chismorreando . <end>
sentence: <start> cada vez que llueve , el techo gotea . <end>
sentence: <start> ¿ podrias echarle un vistazo a este documento ? <end>
sentence: <start> tom cerro la boca . <end>
sentence: <start> estoy de vuelta en boston . <end>
sentence: <start> mary es una persona calida y bondadosa , pero tom es frio y distante . <end>
no words!
sentence: <start> ¿ por que no vas en mi lugar ? <end>
sentence: <start> ¿ me estas bromeando ? <end>
sentence: <start> ella canto suavemente la cancion . <end>
sentence: <start> ¿ a quien esta buscando tom ? <end>
sentence: <start> este edificio esta hecho de piedra . <end>
sentence: <start> me gustaria abrir una cuenta de ahorros . <end>
sentence: <start> ella le compro un coche , pero el no tenia carnet de conducir , de modo que no podia llevarlo a ninguna parte . <end>
sentence: <start> tan pronto como pueda costea

sentence: <start> tom no sabe nada acerca del pasado de mary . <end>
sentence: <start> las matematicas son una materia facil para mi . <end>
sentence: <start> el levanto la mano para hacer una pregunta . <end>
sentence: <start> yo no tengo nada que ver con ese crimen . <end>
sentence: <start> comence a escribir un libro . <end>
sentence: <start> algo asi no puede ocurrir en japon . <end>
sentence: <start> tomas abrio la jaula . <end>
sentence: <start> olvide que hoy era sabado . <end>
sentence: <start> el gato es cafe . <end>
sentence: <start> el sudor permite regular la temperatura corporal . <end>
sentence: <start> te mostrare la ciudad . <end>
sentence: <start> debe su exito a sus padres . <end>
sentence: <start> esa no es mi linea . <end>
sentence: <start> pague la cuenta . <end>
sentence: <start> es hora de que los ninos vayan a la cama . <end>
sentence: <start> tom bebe como un pez . <end>
sentence: <start> ella me saludo con una sonrisa . <end>
sentence: <start> todavia no hemos

sentence: <start> tom estuvo todo el dia reunido . <end>
sentence: <start> no estoy acostumbrada a hablar en publico . <end>
sentence: <start> tom no esta muy feliz . <end>
sentence: <start> tengo que tomar algo para mi resfriado . <end>
sentence: <start> elige con cuidado . <end>
sentence: <start> eres una linda chica . <end>
sentence: <start> tom queria cambiar el tema . <end>
sentence: <start> quiero hablarte , tom . <end>
sentence: <start> nadie me escucha . <end>
sentence: <start> esto se ve estupendo . <end>
sentence: <start> estoy estudiando ingles . <end>
sentence: <start> tom volvio a acomodarse en su camion . <end>
no words!
sentence: <start> ella es una profesora muy buena . <end>
sentence: <start> tom adivino bien . <end>
no words!
sentence: <start> tom quiere saber cuando mary ira de compras . <end>
sentence: <start> ustedes tres estan arrestadas . <end>
sentence: <start> tom se hizo medico . <end>
sentence: <start> aquellos que viven en casas de cristal no deberian tirar 

sentence: <start> vamos a tener un bebe . <end>
sentence: <start> tom parece preocupado . <end>
sentence: <start> continue su analisis . <end>
no words!
sentence: <start> el suele ser arrogante . <end>
sentence: <start> tom tiene un fuerte resfriado . <end>
sentence: <start> tom no tiene tiempo para ayudarte . <end>
sentence: <start> querria un poco de queso . <end>
sentence: <start> no podria haberlo expresado mejor . <end>
sentence: <start> ¿ por que estas tan seguro de que tom es canadiense ? <end>
sentence: <start> el maestro le leyo un parrafo de la biblia a la clase . <end>
sentence: <start> tom fue herido . <end>
sentence: <start> tu tienes prohibido fumar aqui . <end>
sentence: <start> quiero estar contigo pase lo que pase . <end>
sentence: <start> todos miramos por la ventana . <end>
sentence: <start> no podia dar otra mordida . <end>
sentence: <start> un doctor puede enterrar sus errores , pero un arquitecto solo puede sugerirle a su cliente plantar enredaderas . <end>
no wor

In [153]:
# translate('¿Tiene bebidas sin alcohol?') 

In [154]:
# translate(u'esta es mi vida.')  # this is my life